## Importing the goodies 

In [4]:
import torch
import torchvision
import torch.nn as nn                                              
import torch.nn.functional as F
import torch.optim as optim
torch.set_printoptions(linewidth = 120)
torch.set_grad_enabled(True)
import torchvision
import matplotlib.pyplot as plt
from IPython.display import clear_output
import numpy as np
import pyautogui
from PIL import Image 
from Xlib import display
import numpy as np
import matplotlib.pyplot as plt
import time
from mss.linux import MSS as mss
import mss.tools
import cv2
import torchvision.transforms as transforms




## Defining the nn layers and loading up the trained model

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 10, 5)
        self.pool = nn.MaxPool2d(3, 3)
        
        self.dropout = nn.Dropout(p=0.5)
        self.conv2 = nn.Conv2d(10, 15, 5)
        
        self.fc1 = nn.Linear(105,70)
        self.fc2 = nn.Linear(70, 50)
        self.fc3 = nn.Linear(50, 1)

    def forward(self, x):
        x = (self.pool(F.relu(self.conv1(x))))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

drunk_driver = Net()
print(drunk_driver)

drunk_driver.load_state_dict(torch.load("torch/drunk_driver.pt"))

Net(
  (conv1): Conv2d(3, 10, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.5, inplace=False)
  (conv2): Conv2d(10, 15, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=105, out_features=70, bias=True)
  (fc2): Linear(in_features=70, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=1, bias=True)
)


<All keys matched successfully>

## Defining some utilities 

In [25]:



def find_mouse(x = False , y = False):
    
    data = display.Display().screen().root.query_pointer()._data
    res = [data["root_x"], data["root_y"]]
    if x == True and y == True:
        return res
    elif x == True and y == False:
        return res[0]
    else:
        return res[1]

def move_mouse_co_ord_x(x_val , time = 0.001):
    
    current_y = find_mouse(y = True)
    pyautogui.moveTo(x_val, current_y, duration= time )
    

def screen_record_efficient(top = 225, left = 0, width = 800, height = 260, resize_width = 80, resize_height = 25):
  
    mon = {"top": top, "left": left, "width": width, "height": height}
    sct = mss.mss()
    
    img = np.array(sct.grab(mon))
    img = cv2.resize(img, (80,25))
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)
    

    sct.close()
    return img

loader = transforms.Compose([transforms.ToTensor()])

def numpy_to_tensor(np_arr):
    
    image = loader(np_arr).float().unsqueeze(0)
    
    return image
   

## And it all comes together here
> * Feel free to change the vertices in `screen_record_efficient()` depending on where you place the game window on the screen
*  Drag mouse to the corner of the screen to quit

In [33]:

def not_so_drunk_driver():
    
    print ("running...")
    print ("drag mouse to the corner of the screen to terminate -")
    while True:
        try:
            foo = screen_record_efficient(top = 335,width = 800, height = 120, resize_width = 80, resize_height = 25)  ## FPV TRACK VIEW
            
            foo = numpy_to_tensor(foo)
            steer = int(drunk_driver(foo).item())
            
#             print(steer)
            x_co = 400 - steer


            move_mouse_co_ord_x(x_co , time = 0)


        except:
          

            print("end ----")
            break

In [35]:
not_so_drunk_driver()    

running...
drag mouse to the corner of the screen to terminate -
end ----


In [18]:
cv2.destroyAllWindows()